In [ ]:
# Imports

import numpy as np
import pandas as pd
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sn
import wandb

In [ ]:
# Creating train and test data

(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
X_train_new = X_train.reshape(
    (X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_test_new = X_test.reshape(
    (X_test.shape[0], X_test.shape[1] * X_test.shape[2]))

In [ ]:
# Logging sample images

wandb.init(project='cs6910-a1')
done = set()
for x, y in zip(X_train, Y_train):
    if y not in done:
        done.add(y)
        wandb.log({'img': wandb.Image(x, caption='Set ' + str(y))})

In [ ]:
# A class that generates a Feedforward Neural Network with given hyperparameters

class FNNClassifier:
    def __init__(self,
                 layer_size,
                 num_layers,
                 activation = 'ReLU',
                 optimizer = 'adam',
                 weight_decay = 0.0001,
                 batch_size = 200,
                 learning_rate = 0.001,
                 num_epochs = 200,
                 weight_init = 'Xavier',
                 loss = 'cross_entropy'):
        self.activation = activation  # 'sigmoid', 'tanh', 'ReLU'
        self.optimizer = optimizer  # 'normal', 'sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam'
        self.weight_decay = weight_decay  # L2 regularization hyperparameter
        self.batch_size = batch_size  # Batch size
        self.learning_rate = learning_rate  # Learning Rate
        self.num_epochs = num_epochs  # Number of epochs
        self.weight_init = weight_init  # 'random', 'Xavier'
        self.loss = loss  # 'cross_entropy', 'square'
        self.n = 100
        self.K = 10
        self.L = num_layers
        self.N = layer_size
        self.layer_sizes = np.zeros((num_layers + 2))
        self.layer_sizes[1:num_layers + 1] = layer_size
        self.W = []
        self.b = []
        self.a = []
        self.h = []

    def act(self, z):
        if (self.activation == 'sigmoid'):
            return np.where(z > 0, 1 / (1 + np.exp(-z)),
                            np.exp(z) / (1 + np.exp(z)))
        elif (self.activation == 'tanh'):
            return np.where(z > 0, (1 - np.exp(-2 * z)) / (1 + np.exp(-2 * z)),
                            (np.exp(2 * z) - 1) / (np.exp(2 * z) + 1))
        elif (self.activation == 'ReLU'):
            return np.maximum(z, np.zeros(z.shape))

    def deriv_act(self, z):
        if (self.activation == 'sigmoid'):
            return self.act(z) * (1 - self.act(z))
        elif (self.activation == 'tanh'):
            return 1 - self.act(z)**2
        elif (self.activation == 'ReLU'):
            return np.maximum(np.sign(z), np.zeros(z.shape))

    def oact(self, z):
        temp = np.exp(z) / np.exp(z).sum(axis=0)
        return np.where(np.isnan(temp).sum(axis=0) == 0, temp, z == z.max(axis=0))

    def loss_calc(self, Y, Y_pred):
        if (self.loss == 'cross_entropy'):
            return -np.sum(np.log2(Y_pred[np.arange(Y.shape[0]), Y]))
        elif (self.loss == 'square'):
            return (np.sum(Y_pred**2) + Y.shape[0] -
                    2 * np.sum(Y_pred[np.arange(Y.shape[0]), Y])) / 2

    def accuracy(self, Y, Y_pred):
        return np.sum(Y_pred == Y) / Y.shape[0]

    def forward_prop(self, X):
        self.a = [np.zeros((1, 1))]
        self.h = [X]

        for i in range(1, self.L + 1):
            self.a.append((self.b[i].T + (self.W[i].T @ self.h[i - 1]).T).T)
            self.h.append(self.act(self.a[i]))
        self.h[-1] = self.oact(self.a[-1])
        return self.h[-1]

    def back_prop(self, Y_pred, ey):
        self.gradW, self.gradB, self.grada = [], [], []
        self.gradh = [np.zeros((1, 1))]
        if (self.loss == 'cross_entropy'):
            self.grada.append(-(ey - Y_pred))
        elif (self.loss == 'square'):
            self.grada.append((ey - Y_pred) * Y_pred -
                              (Y_pred @ (ey - Y_pred).T) @ Y_pred)
        for i in range(self.L, 0, -1):
            self.gradW.append((self.grada[self.L - i] @ self.h[i - 1].T).T)
            self.gradB.append(self.grada[self.L - i].sum(axis=1))
            self.gradh.append(self.W[i] @ self.grada[self.L - i])
            self.grada.append(self.gradh[self.L - i + 1] *
                              self.deriv_act(self.a[i - 1]))

        self.gradW.append(np.zeros((self.N, self.N)))
        self.gradB.append(np.zeros(self.N))

        return

    def wb_init(self, num):
        if (self.weight_init == 'random'):
            for i in range(self.L):
                self.W.append(np.random.randn(self.N, self.N))
                self.b.append(np.random.randn(self.N))
            self.W.append(np.random.randn(self.N, self.K))
            self.W[1] = np.random.randn(num, self.W[1].shape[1])
            self.b.append(np.random.randn(self.K))

        elif (self.weight_init == 'Xavier'):
            for i in range(self.L):
                self.W.append(
                    np.random.normal(0, np.sqrt(1 / self.N), (self.N, self.N)))
                self.b.append(np.random.normal(0, np.sqrt(1 / self.N), self.N))
            self.W.append(
                np.random.normal(0, np.sqrt(2 / (self.N + self.K)),
                                 (self.N, self.K)))
            self.W[1] = np.random.normal(
                0, np.sqrt(2 / (num + self.W[1].shape[1])),
                (num, self.W[1].shape[1]))
            self.b.append(np.random.normal(0, np.sqrt(1 / self.K), self.K))

    def grad_desc(self, X, Y, X_val = 0, Y_val = 0, validate = False):

        self.wb_init(X.shape[0])

        W_0, b_0 = [], []
        for i in range(self.L):
            W_0.append(np.zeros((self.N, self.N)))
            b_0.append(np.zeros(self.N))
        W_0.append(np.zeros((self.N, self.K)))
        W_0[1] = np.zeros((X.shape[0], self.W[1].shape[1]))
        b_0.append(np.zeros(self.K))

        update_W, update_b = W_0, b_0
        v_W, v_b = W_0, b_0
        m_W, m_b = W_0, b_0
        W_original, b_original = [], []

        ey = np.zeros((self.K, self.n))
        rows = np.arange(self.n)
        ey[Y.T, rows] = 1

        for t in range(1, self.num_epochs + 1):

            # wandb.log({'epoch': t})

            if (self.optimizer == 'sgd'):
                for tt in range(
                        0,
                    ((self.n + self.batch_size - 1) // self.batch_size)):
                    idx = np.random.randint(self.n, size=self.batch_size)
                    ey = np.zeros((self.K, self.batch_size))
                    rows = np.arange(self.batch_size)
                    ey[Y.T[idx], rows] = 1
                    Y_pred = self.forward_prop(X.T[idx, :].T)

                    self.back_prop(Y_pred, ey)
                    dW = self.gradW[::-1]
                    db = self.gradB[::-1]

                    update_W = [self.learning_rate * u for u in dW]
                    update_b = [self.learning_rate * u for u in db]
                    update_W = [
                        u + v * self.weight_decay
                        for u, v in zip(update_W, dW)
                    ]
                    self.W = [u - v for u, v in zip(self.W, update_W)]
                    self.b = [u - v for u, v in zip(self.b, update_b)]

                if validate:
                    self.validate(X.T, Y.T, X_val, Y_val)
                continue

            Y_pred = self.forward_prop(X)

            if (self.optimizer == 'nesterov'):
                W_original, b_original = self.W, self.b
                ngamma = 0.5  # hyperparameter
                self.W = [u - ngamma * v for u, v in zip(self.W, update_W)]
                self.b = [u - ngamma * v for u, v in zip(self.b, update_b)]

            self.back_prop(Y_pred, ey)

            dW = self.gradW[::-1]
            db = self.gradB[::-1]

            if (self.optimizer == 'normal'):
                update_W = [self.learning_rate * u for u in dW]
                update_b = [self.learning_rate * u for u in db]

            elif (self.optimizer == 'momentum'):
                mgamma = 0.5  # hyperparameter
                update_W = [
                    mgamma * u + self.learning_rate * v
                    for u, v in zip(update_W, dW)
                ]
                update_b = [
                    mgamma * u + self.learning_rate * v
                    for u, v in zip(update_b, db)
                ]

            elif (self.optimizer == 'nesterov'):
                self.W, self.b = W_original, b_original
                ngamma = 0.5  # hyperparameter
                update_W = [
                    ngamma * u + self.learning_rate * v
                    for u, v in zip(update_W, dW)
                ]
                update_b = [
                    ngamma * u + self.learning_rate * v
                    for u, v in zip(update_b, db)
                ]

            elif (self.optimizer == 'rmsprop'):
                rbeta = 0.9
                epsilon = 10**-8
                v_W = [
                    rbeta * u + (1 - rbeta) * (v**2) for u, v in zip(v_W, dW)
                ]
                v_b = [
                    rbeta * u + (1 - rbeta) * (v**2) for u, v in zip(v_b, db)
                ]
                update_W = [(self.learning_rate * u) / np.sqrt(v + epsilon)
                            for u, v in zip(dW, v_W)]
                update_b = [(self.learning_rate * u) / np.sqrt(v + epsilon)
                            for u, v in zip(db, v_b)]

            elif (self.optimizer == 'adam'):
                abeta1 = 0.9
                abeta2 = 0.999
                epsilon = 10**-8
                m_W = [abeta1 * u + (1 - abeta1) * v for u, v in zip(m_W, dW)]
                m_b = [abeta1 * u + (1 - abeta1) * v for u, v in zip(m_b, db)]
                v_W = [
                    abeta2 * u + (1 - abeta2) * (v**2)
                    for u, v in zip(v_W, dW)
                ]
                v_b = [
                    abeta2 * u + (1 - abeta2) * (v**2)
                    for u, v in zip(v_b, db)
                ]
                update_W = [((self.learning_rate * u) / (1 - abeta1**t)) /
                            np.sqrt((v / (1 - abeta2**t)) + epsilon)
                            for u, v in zip(m_W, v_W)]
                update_b = [((self.learning_rate * u) / (1 - abeta1**t)) /
                            np.sqrt((v / (1 - abeta2**t)) + epsilon)
                            for u, v in zip(m_b, v_b)]

            elif (self.optimizer == 'nadam'):
                nbeta1 = 0.9
                nbeta2 = 0.999
                epsilon = 10**-8
                m_W = [nbeta1 * u + (1 - nbeta1) * v for u, v in zip(m_W, dW)]
                m_b = [nbeta1 * u + (1 - nbeta1) * v for u, v in zip(m_b, db)]
                v_W = [
                    nbeta2 * u + (1 - nbeta2) * (v**2)
                    for u, v in zip(v_W, dW)
                ]
                v_b = [
                    nbeta2 * u + (1 - nbeta2) * (v**2)
                    for u, v in zip(v_b, db)
                ]
                update_W = [(self.learning_rate *
                             (nbeta1 * u + (1 - nbeta1) * v)) / (1 - nbeta1**t)
                            for u, v in zip(m_W, dW)]
                update_W = [(u) / np.sqrt((v / (1 - nbeta2**t)) + epsilon)
                            for u, v in zip(update_W, v_W)]
                update_b = [(self.learning_rate *
                             (nbeta1 * u + (1 - nbeta1) * v)) / (1 - nbeta1**t)
                            for u, v in zip(m_b, db)]
                update_b = [(u) / np.sqrt((v / (1 - nbeta2**t)) + epsilon)
                            for u, v in zip(update_b, v_b)]

            update_W = [
                u + v * self.weight_decay for u, v in zip(update_W, dW)
            ]

            self.W = [u - v for u, v in zip(self.W, update_W)]
            self.b = [u - v for u, v in zip(self.b, update_b)]

            if validate:
                self.validate(X.T, Y.T, X_val, Y_val)

    def validate(self, X_tr, Y_tr, X_val, Y_val):
        Y_pred_proba = self.predict_proba(X_tr)
        Y_pred = Y_pred_proba.argmax(axis=1)
        # wandb.log({'loss': self.loss_calc(Y_tr, Y_pred_proba)})
        # wandb.log({'accuracy': self.accuracy(Y_tr, Y_pred)})

        Y_pred_val_proba = self.predict_proba(X_val)
        Y_pred_val = Y_pred_val_proba.argmax(axis=1)
        # wandb.log({'val_loss': self.loss_calc(Y_val, Y_pred_val_proba)})
        # wandb.log({'val_accuracy': self.accuracy(Y_val, Y_pred_val)})

    def fit_validate(self, X_train, Y_train):
        X_tr, X_val, Y_tr, Y_val = train_test_split(X_train,
                                                    Y_train,
                                                    test_size=0.5,
                                                    random_state=42)
        self.n = X_tr.shape[0]
        self.batch_size = min(self.batch_size, self.n)
        K = np.max(Y_tr) + 1  # Y_train must have values from 0 to K - 1
        self.layer_sizes[self.L + 1] = K
        self.K = K
        self.grad_desc(X_tr.T, Y_tr.T, X_val, Y_val, True)

    def fit(self, X_train, Y_train):
        self.n = X_train.shape[0]
        self.batch_size = min(self.batch_size, self.n)
        K = np.max(Y_train) + 1  # Y_train must have values from 0 to K - 1
        self.layer_sizes[self.L + 1] = K
        self.K = K
        self.grad_desc(X_train.T, Y_train.T)

    def predict_proba(self, X_test):
        return self.forward_prop(X_test.T).T

    def predict(self, X_test):
        return (self.predict_proba(X_test).argmax(axis=1))


In [ ]:
# A login cell to wandb

!wandb login --relogin 2bb9d168b199eb6ac3f9f3890628908fdc41ffd9

In [ ]:
# Setting up parameters for the sweep

sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'layer_size': {
            'values': [32, 64, 128]
        },
        'num_layers': {
            'values': [3, 4, 5]
        },
        'activation': {
            'values': ['ReLU', 'sigmoid', 'tanh']
        },
        'optimizer': {
            'values':
            ['sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam']
        },
        'weight_decay': {
            'values': [0, 0.0005, 0.5]
        },
        'batch_size': {
            'values': [16, 32, 64]
        },
        'num_epochs': {
            'values': [5, 10]
        },
        'learning_rate': {
            'values': [0.01, 0.001, 0.0001]
        },
        'weight_init': {
            'values': ['random', 'Xavier']
        }
    }
}


In [ ]:
# Setting up a sweep

sweep_id = wandb.sweep(sweep_config, entity = '0x2e4', project = 'cs6910-a1')

In [ ]:
# A training function for the sweep

def train():
    def_params = {
        'layer_size': 32,
        'num_layers': 4,
        'activation': 'ReLU',
        'optimizer': 'adam',
        'weight_decay': 0.0005,
        'batch_size': 32,
        'num_epochs': 5,
        'learning_rate': 0.001,
        'weight_init': 'random'
    }
    run = wandb.init(config=def_params, reinit=True)
    config = wandb.config
    wandb.run.name = str(config.layer_size) + '_' + str(config.num_layers) + '_' + config.activation[0] + '_' + config.optimizer[:2] + '_' + str(config.weight_decay) + '_' + str(config.batch_size) + '_' + str(config.num_epochs) + '_' + str(config.learning_rate) + '_' + config.weight_init[0]
    wandb.run.save()
    model = FNNClassifier(**config)
    model.fit_validate(X_train_new, Y_train)

In [ ]:
# Setting up a sweep agent

wandb.agent(sweep_id, train)

In [ ]:
# Creating best fit FNN for Fashion - MNIST

best_model_entropy = FNNClassifier(64, 5, 'ReLU', 'sgd', 0, 16, 0.0001, 10, 'Xavier')
best_model_entropy.fit(X_train_new, Y_train)
Y_pred = best_model_entropy.predict(X_test_new)

In [ ]:
# Creating a confusion matrix on test data

conf_mat = np.zeros((10, 10), dtype = np.int64)
for i in range(Y_pred.shape[0]):
    conf_mat[Y_pred[i]][Y_test[i]] += 1

In [ ]:
# Logging the confusion matrix 

plt.figure(figsize = (10, 10))
v = sn.heatmap(conf_mat, cmap = 'YlGnBu', annot = True, fmt = 'd')
wandb.log({'Confusion matrix' : wandb.Image(v)})